In [1]:
import optuna
from xgboost import XGBClassifier
import pandas as pd
import numpy as np
from sklearn.preprocessing import FunctionTransformer
from sklearn.model_selection import RandomizedSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score

pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

iskaggle = os.environ.get('KAGGLE_KERNEL_RUN_TYPE', '')
if iskaggle:
    path = Path('../input/playground-series-s3e23')
    orig_path = Path('../input/software-defect-prediction')
else:
    path = Path('playground-series-s3e23')
    orig_path = Path('software-defect-prediction')
    if not path.exists():
        kaggle.api.competition_download_cli(str(path))
        zipfile.ZipFile(f'{path}.zip').extractall(path)
    if not orig_path.exists():
        kaggle.api.competition_download_cli(str(path))
        zipfile.ZipFile(f'{path}.zip').extractall(path)

In [2]:
train = pd.read_csv(path/'train.csv', index_col = 'id')
test = pd.read_csv(path/'test.csv', index_col = 'id')
orig_train = pd.read_csv(orig_path/'jm1.csv')

In [3]:
for object_features in list(orig_train.loc[:, orig_train.dtypes == 'O']):
    orig_train[object_features] = orig_train[object_features].replace({'?' : np.nan}).astype('float64')

In [4]:
X = pd.concat([train, orig_train])
y = X.pop('defects')

seed = 42
splits = 5
skf = StratifiedKFold(n_splits = splits, random_state = seed, shuffle = True)
np.random.seed(seed)

In [5]:
log_transformer = FunctionTransformer(func=np.log1p, validate=False)

In [ ]:
partial_pipeline = make_pipeline(SimpleImputer(), log_transformer, StandardScaler())

In [8]:
def objective(trial):
    params = {
        'objective': 'binary:logistic',
        'tree_method':'gpu_hist',  # this parameter means using the GPU when training our model to speedup the training process
        'n_jobs': trial.suggest_categorical('n_jobs', [-1]),
        'random_state': trial.suggest_categorical('random_state', [seed]),
        'lambda': trial.suggest_float('lambda', 1e-3, 10.0, log=True),
        'alpha': trial.suggest_float('alpha', 1e-3, 10.0, log=True),
        'colsample_bytree': trial.suggest_categorical('colsample_bytree', [0.3,0.4,0.5,0.6,0.7,0.8,0.9, 1.0]),
        'subsample': trial.suggest_categorical('subsample', [0.4,0.5,0.6,0.7,0.8,1.0]),
        'learning_rate': trial.suggest_categorical('learning_rate', [0.008,0.01,0.012,0.014,0.016,0.018, 0.02]),
        'n_estimators': trial.suggest_int('n_estimators', 100, 600, 100),
        'max_depth': trial.suggest_categorical('max_depth', [5,7,9,11,13,15,17]),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 300),
    }

    pipeline = make_pipeline(partial_pipeline, XGBClassifier(**params))
    scores = cross_val_score(pipeline, X, y, cv=3, scoring='roc_auc', n_jobs=-1)
    return scores.mean()

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=15, show_progress_bar=True)

[I 2023-10-12 18:36:40,248] A new study created in memory with name: no-name-59e05e7a-8026-48fd-99d9-99e36dff9805
  0%|                                                                                                                                                                                                                 | 0/15 [00:00<?, ?it/s]/home/strawhatdragon100/miniconda3/envs/kaggle/lib/python3.11/site-packages/xgboost/data.py:440: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):
/home/strawhatdragon100/miniconda3/envs/kaggle/lib/python3.11/site-packages/xgboost/data.py:440: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):
/home/strawhatdragon100/miniconda3/envs/kaggle/lib/python3.11/site-packages/xgboost/data.py:440: FutureWarning: is_sparse is deprecated and will be r

[I 2023-10-12 18:41:58,927] Trial 0 finished with value: 0.7802572415437462 and parameters: {'lambda': 0.038721308691319005, 'alpha': 0.002234181988950396, 'colsample_bytree': 0.3, 'subsample': 0.6, 'learning_rate': 0.01, 'max_depth': 11, 'random_state': 2020, 'min_child_weight': 181}. Best is trial 0 with value: 0.7802572415437462.


/home/strawhatdragon100/miniconda3/envs/kaggle/lib/python3.11/site-packages/xgboost/data.py:440: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):
/home/strawhatdragon100/miniconda3/envs/kaggle/lib/python3.11/site-packages/xgboost/data.py:440: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):
/home/strawhatdragon100/miniconda3/envs/kaggle/lib/python3.11/site-packages/xgboost/data.py:440: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):
Best trial: 0. Best value: 0.780257:  13%|█████████████████████▋                                                                                                                                             | 2/15 [08:54<55:55, 258.10s/it]

[I 2023-10-12 18:45:34,620] Trial 1 finished with value: 0.7721428013359716 and parameters: {'lambda': 0.6834867117482265, 'alpha': 0.12267535284709595, 'colsample_bytree': 0.6, 'subsample': 0.6, 'learning_rate': 0.018, 'max_depth': 5, 'random_state': 2020, 'min_child_weight': 25}. Best is trial 0 with value: 0.7802572415437462.


/home/strawhatdragon100/miniconda3/envs/kaggle/lib/python3.11/site-packages/xgboost/data.py:440: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):
/home/strawhatdragon100/miniconda3/envs/kaggle/lib/python3.11/site-packages/xgboost/data.py:440: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):
/home/strawhatdragon100/miniconda3/envs/kaggle/lib/python3.11/site-packages/xgboost/data.py:440: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):
Best trial: 0. Best value: 0.780257:  20%|████████████████████████████████▌                                                                                                                                  | 3/15 [14:30<58:42, 293.53s/it]

...

[I 2023-10-12 19:26:30,931] Trial 12 finished with value: 0.781638096898329 and parameters: {'lambda': 9.536789257111334, 'alpha': 0.0010880289210026032, 'colsample_bytree': 0.3, 'subsample': 0.4, 'learning_rate': 0.012, 'max_depth': 17, 'random_state': 2020, 'min_child_weight': 194}. Best is trial 11 with value: 0.7822810650792745.


/home/strawhatdragon100/miniconda3/envs/kaggle/lib/python3.11/site-packages/xgboost/data.py:440: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):
/home/strawhatdragon100/miniconda3/envs/kaggle/lib/python3.11/site-packages/xgboost/data.py:440: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):
/home/strawhatdragon100/miniconda3/envs/kaggle/lib/python3.11/site-packages/xgboost/data.py:440: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):
Best trial: 11. Best value: 0.782281:  93%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▎          | 14/15 [52:26<02:47, 167.17s/it]

[I 2023-10-12 19:29:07,153] Trial 13 finished with value: 0.7819635680288797 and parameters: {'lambda': 8.998327524304006, 'alpha': 0.011142578300533888, 'colsample_bytree': 1.0, 'subsample': 0.4, 'learning_rate': 0.008, 'max_depth': 13, 'random_state': 2020, 'min_child_weight': 196}. Best is trial 11 with value: 0.7822810650792745.


/home/strawhatdragon100/miniconda3/envs/kaggle/lib/python3.11/site-packages/xgboost/data.py:440: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):
/home/strawhatdragon100/miniconda3/envs/kaggle/lib/python3.11/site-packages/xgboost/data.py:440: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):
/home/strawhatdragon100/miniconda3/envs/kaggle/lib/python3.11/site-packages/xgboost/data.py:440: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):
Best trial: 11. Best value: 0.782281: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 15/15 [54:48<00:00, 219.20s/it]

[I 2023-10-12 19:31:28,261] Trial 14 finished with value: 0.7819319954355936 and parameters: {'lambda': 2.0129141475638384, 'alpha': 0.006031039568808718, 'colsample_bytree': 0.4, 'subsample': 0.8, 'learning_rate': 0.01, 'max_depth': 7, 'random_state': 2020, 'min_child_weight': 298}. Best is trial 11 with value: 0.7822810650792745.


In [9]:
study.best_params, study.best_value

({'lambda': 6.92479678295337,
  'alpha': 0.0016970742112315522,
  'colsample_bytree': 0.3,
  'subsample': 0.4,
  'learning_rate': 0.01,
  'max_depth': 11,
  'random_state': 2020,
  'min_child_weight': 193},
 0.7822810650792745)

In [10]:
pipeline = make_pipeline(SimpleImputer(), log_transformer, XGBClassifier(**study.best_params))
pipeline.fit(X, y)

/home/strawhatdragon100/miniconda3/envs/kaggle/lib/python3.11/site-packages/xgboost/data.py:440: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):


Pipeline(steps=[('simpleimputer', SimpleImputer()),
                ('functiontransformer',
                 FunctionTransformer(func=<ufunc 'log1p'>)),
                ('xgbclassifier',
                 XGBClassifier(alpha=0.0016970742112315522, base_score=None,
                               booster=None, callbacks=None,
                               colsample_bylevel=None, colsample_bynode=None,
                               colsample_bytree=0.3, early_stopping_rounds=None,
                               enable_categorical=False, eval_metric=None,
                               feature_types=None, gamma=None, gpu_id=None,
                               grow_policy=None, importance_type=None,
                               interaction_constraints=None,
                               lambda=6.92479678295337, learning_rate=0.01,
                               max_bin=None, max_cat_threshold=None,
                               max_cat_to_onehot=None, max_delta_step=None,
                               max_depth=11, max_leaves=None,
                               min_child_weight=193, missing=nan,
                               monotone_constraints=None, n_estimators=100,
                               n_jobs=None, num_parallel_tree=None, ...))])

In [11]:
submission = test.copy()
submission['defects'] = pipeline.predict_proba(submission)[:, 1]
submission.defects.to_csv('xgboost_submission.csv')